In [30]:
from datasets import load_dataset
ds = load_dataset("RUC-AIBOX/STILL-3-Preview-RL-Data")
# Convert ds into json list of dicotinaries
still_ds = ds['train'].map(lambda x: {"problem": x["question"], "answer": x["answer"],})


In [ ]:
# Load all jsons in ../test and merge them into one large json
import os
import json

prior_data = []
for file in ['aime.json', 'amc.json', 'math.json', 'omni_math.json']:
    # get absolute path by merging file wtih directory
    file_path = os.path.join("../train", file)
    file_path = os.path.abspath(file_path)
    with open(file_path, "r") as f:
        data = json.load(f)
    prior_data.extend(data)
    
len(prior_data)


In [27]:
from deepscaler.utils import RAG

rag_searcher = RAG(docs=[d["problem"] for d in prior_data])

In [ ]:
# Filter for olympiad problems that are not in the omni dataset
from tqdm import tqdm

filter_problems = []
num_problems = 0

counter = 0
# Wrap olympiad_data with tqdm, optionally adding a description and total
for d in tqdm(still_ds, desc="Filtering stil data", total=len(still_ds)):
    search_result = rag_searcher.top_k(d["problem"], k=1)[0]
    score = search_result["score"]
    if score > 0.90:
        num_problems += 1
    else:
        filter_problems.append(d)
    counter += 1
    if counter %1000 == 0:
        print(counter)
# Save final list as json
with open("still.json", "w") as f:
    json.dump(filter_problems, f, indent=2)

In [ ]:
for d in filter_problems:
    del d["question"]
    del d['messages']

# Save final list as json
with open("still.json", "w") as f:
    json.dump(filter_problems, f, indent=2)
len(filter_problems)